In [1]:
from __future__ import absolute_import, division, print_function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import collections
import os
import string
import time

In [2]:
import keras
import sklearn
import tensorflow as tf

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

Using TensorFlow backend.


In [4]:
df = pd.read_csv('data/kaggle/shortjokes.csv')
df = df.loc[df["Joke"] != ""]

In [140]:
def clean_data(lst, in_place=True, keep_punc=""):
    if in_place:
        new_lst = lst
    else:
        new_lst = lst.copy()
        
    for i in range(len(new_lst)):
        joke = lst[i]
        joke = joke.replace("\r", " ").replace("\n", " ").replace("/", " or ")
        joke = joke.translate(str.maketrans("", "", string.punctuation.replace(keep_punc, ""))) # Remove punctuation
        joke = joke.lower() # Lowercase
        joke = ''.join(char for char in joke if not char.isdigit()) # Remove numbers
        joke = joke.strip() # Remove leading and ending whitespace
        new_lst[i] = joke
    return new_lst

In [9]:
data = clean_data(df["Joke"].tolist(), keep_punc="?")

#### Filter by QA jokes

In [63]:
qa = [[s.strip() for s in list(filter(None, joke.split("?")))] for joke in data if len(list(filter(None, joke.split("?")))) == 2]

#### Only keep jokes with a question length > 24 and an answer length > 4 

In [64]:
qa = [(q, a) for q, a in qa if len(q) >= 25 and len(a) >= 5 ]

In [70]:
combined_qa = [q + " " + a for q, a in qa]

In [130]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_qa)
word2idx = tokenizer.word_index.copy()
idx2word = {key: value for value, key in word2idx.items()}
vocab_size = len(word2idx)
print("Vocab size:", vocab_size)

Vocab size: 44464


In [131]:
encoded = tokenizer.texts_to_sequences(combined_qa)
encoded_q, encoded_a = [], []

for i in range(len(encoded)):
    e = encoded[i]
    q_len = len(qa[i][0].split())
    e_q = e[:q_len]
    e_a = e[q_len:]
    encoded_q.append(e_q)
    encoded_a.append(e_a)

encoded_q=keras.preprocessing.sequence.pad_sequences(encoded_q, padding="post")
encoded_a=keras.preprocessing.sequence.pad_sequences(encoded_a, padding="post")

In [139]:
y = tf.keras.utils.to_categorical(encoded_a, num_classes=vocab_size) # need to one-hot labels for training

MemoryError: 

# Create Model